In [1]:
import numpy as np
import pandas as pd

In [2]:
from google.colab import drive

In [3]:
drive.mount('datasets')

Mounted at datasets


In [4]:
df_processed = pd.read_csv('datasets/MyDrive/datasets/bl_property_processed.csv')

In [5]:
!!pip install -U sentence-transformers

['Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/',
 'Collecting sentence-transformers',
 '  Downloading sentence-transformers-2.2.2.tar.gz (85 kB)',
 '\x1b[?25l     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b0.0/86.0 kB\x1b \x1b?\x1b eta \x1b-:--:--\x1b',
 '\x1b[2K     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b86.0/86.0 kB\x1b \x1b3.3 MB/s\x1b eta \x1b0:00:00\x1b',
 '\x1b[?25h  Preparing metadata (setup.py) ... \x1b[?25l\x1b[?25hdone',
 'Collecting transformers<5.0.0,>=4.6.0 (from sentence-transformers)',
 '  Downloading transformers-4.29.2-py3-none-any.whl (7.1 MB)',
 '\x1b[?25l     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b0.0/7.1 MB\x1b \x1b?\x1b eta \x1b-:--:--\x1b',
 '\x1b[2K     \x1b━━━━━\x1b\x1b╺\x1b\x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b0.9/7.1 MB\x1b \x1b28.6 MB/s\x1b eta \x1b0:00:01\x1b',
 '\x1b[2K     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━\x1b\x1b╺\x1b\x1b━━━━━━━━━━━━━━\x1b \x1b4.4/7.1 MB\x1b 

In [6]:
df_processed['recommend_text'][0]

'This 8703254 square ft Area of the Sale Apartment is a residential property with 17520 bedrooms and 16825 baths and priced at BDT 45154079350 in Dhanmondi and zone in Dhanmondi city is Dhaka .  A Vibrant 2464 Sq Ft Residential Flat For Sale In ShukrabadReady to move in somewhere with everything nearby? This apartment bears all the necessary facilities that you look for in a flat and more. There are many facilities in the flat that are required to live at ease. Utility supplies are always available and security services are present at all times for ensuring the safety of the inhabitants. The washroom and kitchen come with all the modern fixtures.  The flat is well secured with the availability of all types of utilities.  Don’t miss this remunerative offer and we are just a call away to close the deal for you!'

In [7]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [8]:
model = SentenceTransformer('sentence-transformers/multi-qa-mpnet-base-dot-v1')

In [9]:
import os 
import numpy as np

descriptions = df_processed['recommend_text'].tolist()
embeddings_file = 'property_recommend_multi-qa-mpnet-base-dot-v1.npy'

In [10]:
if not os.path.exists(embeddings_file):
    description_embeddings = model.encode(descriptions)
    np.save(embeddings_file, description_embeddings)
else:
    # Load the saved embeddings
    description_embeddings = np.load(embeddings_file)

In [11]:
# Define the query and calculate its embedding
query = "Provide 1000 sq feet apartments with 2 beds priced at 500,000 in Gulshan?"
query_embedding = model.encode([query])

In [12]:
# Compute cosine similarity between the query embedding and all description embeddings
similarities = cosine_similarity(query_embedding, description_embeddings)

In [13]:
# Find the indices of the top 5 most similar descriptions
top_n = 5
top_n_indices = similarities[0].argsort()[-top_n:][::-1]
print(top_n_indices)

[4886  341 5515 2481 5651]


In [14]:
# Print the top 5 most similar product descriptions
# Print the top 5 most similar product descriptions
for index in top_n_indices:
    print(f"Property ID: {df_processed.iloc[index]}")

Property ID: area                                                               3600.0
building_type                                                   Apartment
building_nature                                               Residential
image_url               https://images-cdn.bproperty.com/thumbnails/43...
num_bath_rooms                                                        5.0
num_bed_rooms                                                         5.0
price                                                            200000.0
property_description    Flat up for rent in Gulshan 2 area, near Manar...
property_overview       The place Gulshan comes to our mind and what w...
property_url            https://www.bproperty.com/en/property/details-...
purpose                                                              Rent
city                                                                Dhaka
locality                                                          Gulshan
division                 